In [2]:
from sqlite3 import connect
from pandas import read_csv, Series

class AnnotationProcessor(object): #to be modified with processor
    def __init__(self, dbPathOrUrl):
        super().__init__(dbPathOrUrl)

    def uploadData(self, annotations): 
        with connect("relational.db") as con:
            con.commit() 
        
        annotations = read_csv("data/annotations.csv", keep_default_na=False, dtype={"id":"string", "body":"string", "target":"string", "motivation":"string"})

        annotations_ids = annotations[["id", "body", "target", "motivation"]]
        annotations_ids = annotations_ids.rename(columns={"id": "annotation"})
        annotations_internal_id = []
        for idx, row in annotations_ids.iterrows():
            annotations_internal_id.append("annotations-" + str(idx))

        annotations_ids.insert(0, "annotationsId", Series(annotations_internal_id, dtype="string"))
        
        image_ids = annotations[["body"]]
        image_internal_id = []
        for idx, row in annotations_ids.iterrows():
            image_internal_id.append("image-" + str(idx))

        image_ids.insert(0, "imageId", Series(image_internal_id, dtype="string"))
        
        partial_iden = annotations[["target"]]
        partial_iden = partial_iden.rename(columns={"target": "id"})
        partial_iden_internal_id = []
        for idx, row in partial_iden.iterrows():
            partial_iden_internal_id.append("IdEn-" + str(idx))

        partial_iden.insert(0, "partialId", Series(partial_iden_internal_id, dtype="string"))
        
        with connect("relational.db") as con:
            annotations_ids.to_sql("annotationsId", con, if_exists="replace", index=False)
            image_ids.to_sql("imageId", con, if_exists="replace", index=False)
            partial_iden.to_sql("id_entityId", con, if_exists="replace", index=False)